## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-10-31-32 +0000,nyt,Edinburgh Fringe in Photos: The Streets Are Alive,https://www.nytimes.com/2025/08/04/theater/edi...
1,2025-08-05-10-23-39 +0000,bbc,More disclosure of suspects' immigration statu...,https://www.bbc.com/news/articles/c8rygx2xpy7o...
2,2025-08-05-10-19-19 +0000,bbc,"US may demand $15,000 deposit for visas",https://www.bbc.com/news/articles/cp9415d53xko...
3,2025-08-05-10-07-00 +0000,wsj,"Stock Market Today: S&P 500 Futures, Dollar In...",https://www.wsj.com/livecoverage/stock-market-...
4,2025-08-05-10-00-00 +0000,latimes,Hiltzik: How the economy went from a 3% annual...,https://www.latimes.com/business/story/2025-08...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,51
82,new,22
84,gaza,15
60,will,12
73,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,123
77,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...,110
113,2025-08-04-23-23-00 +0000,wsj,President Trump’s decision to fire the head of...,https://www.wsj.com/economy/real-strains-insid...,101
280,2025-08-04-13-01-22 +0000,bbc,Hundreds of Israeli ex-officials appeal to Tru...,https://www.bbc.com/news/articles/crkznje8nz8o...,96
231,2025-08-04-16-50-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,94


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,123,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
77,59,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...
148,57,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
9,50,2025-08-05-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Israel’s new plan f...,https://nypost.com/2025/08/05/us-news/ny-postc...
251,49,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
163,47,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...
157,46,2025-08-04-21-20-09 +0000,nypost,NYC’s iconic Flatiron Building will get bathed...,https://nypost.com/2025/08/04/business/nycs-ic...
87,40,2025-08-05-01-03-45 +0000,nypost,Clip of Sydney Sweeney’s expert shooting range...,https://nypost.com/2025/08/04/us-news/clip-of-...
92,37,2025-08-05-00-50-22 +0000,nypost,Oil baron CEO claims he was subject of ‘sham’ ...,https://nypost.com/2025/08/04/us-news/oil-baro...
293,37,2025-08-04-11-07-32 +0000,nypost,Firebrand Rep. Nancy Mace launches campaign fo...,https://nypost.com/2025/08/04/us-news/nancy-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
